In [ ]:
# In this notebook, qw do extensive experiments to check if the
# properties of the linear programs we have proposed for peer review 
#evaluation

In [30]:
using NBInclude , Plots, StatsBase , Arpack, Statistics, Random, Distributions,LinearAlgebra, Base
using JuMP, MosekTools, SparseArrays, Gurobi, IterTools 


In [ ]:
# meta parameters:
#n: number of students
#q: number of questions a student creates, per round. In real life scenario, the number
#w: vector of students quality 
#k: number of people correctly answering a question
#l: number of people taking a question.

In [44]:
#Function to create a matrix containing the probability P(i>j) of student
#i answering correctly a question made by j. We use the RLT model, and 
#for this function we asume no dependance of question quality: answering
# a question well or not is simply a function of the student quality w.
#@param w: vector of students quality, vector has length n, w in [0,1].
function createAnswerProbabilitiesBLT(w)
    dim= length(w)
    Q = zeros(Float64,(dim, dim))
    for i in 1:dim
        for j in 1: dim
          Q[i,j]= 1/(1+exp(-(w[i]-w[j])))  
        end
    end
    return (Q)
end

createAnswerProbabilitiesBLT (generic function with 1 method)

In [43]:
#function that returns a dictionary containing the questions that
# each student answered. Keys are students.
# @param n number of students.
# @param m number of questions. m is a multiple of n.
# @param number_questions_taken number of questions each student takes
function createAnsweredQuestions(n,m,number_questions_taken)
        dict = Dict()   
    for i in 1:n      
    questions = collect(1:m)
        sampled = sample( deleteat!(questions, findall(x->x==i,questions)),number_questions_taken,replace=false )   
       # return(dict)
        dict[i] =  sampled
    end
    return(dict)
end

createAnsweredQuestions (generic function with 1 method)

In [33]:
answered = createAnsweredQuestions(5,5,3)

Dict{Any, Any} with 5 entries:
  5 => [4, 1, 3]
  4 => [2, 5, 3]
  2 => [5, 4, 3]
  3 => [1, 5, 4]
  1 => [4, 2, 3]

In [55]:
#Function to create a student-question matrix. we have n students
# and m questions. We assume m is a multiple of n. 
# te matrix contains a 1 if the student answered correctly the question.
#@studentQuality: underliying quality of the students.
#param takenQuestions dictionary that indicates which questions student i takes.
# n = length(studentQuality)
#@ returns a matrix M where M_ij denotes if student i answered question
#j correctly. 
# For now, we ignore the question difficulty, and simply set the probability 
#of answerng a question correctly as the quality w of a student
# We assume each students takes questions_taken questions 1=<questions_taken<=n
function sampleStudentQuestionMatrix(m,number_questions_taken,questionsTaken,studentQuality)
    n = length(studentQuality)
    M = zeros(n,m)
    for i in 1:n
       
         for j in questionsTaken[i]      
            
            got_it_correctly = rand(Binomial(1,studentQuality[i]),1)[1]
            if(got_it_correctly == 1)
            
            M[i,j] = 1
            else 
                M[i,j] = -1
            end
             
            
        end
    end
    return(M)
end

sampleStudentQuestionMatrix (generic function with 2 methods)

In [62]:
# compute pairwise comparaisons
# for each student, we compute the comparaisons between questions.
# we assume that all of the questions can be compared.
# the probability of comparing favourably is given by the BTL model,
# which we have computed in createAnswerProbabilitiesBLT.
#m the total number of questions
#@param questions_taken a dictionary of student- vectors of questions taken by each student.
# @ returns a dictionary of student- ordered pairs , indicating which question was better ranked by student i

function computeOrderedPairs(questions_taken,number_questions_taken,m,BLTMatrix)

    comparaisons = Dict()
    n = size(BLTMatrix)[1]
    for i in 1:n
    matrixResult = zeros(m,m)
        currentQuestions = get(questions_taken,i,0)
    
      question_pairs =  collect(subsets(currentQuestions,2)) 
      for j in question_pairs
           sampleProbability =  BLTMatrix[j[1],j[2]]
           sampleOutcome = rand(Binomial(1,sampleProbability),1)[1] 
        if(sampleOutcome== 1)
                matrixResult[j[1],j[2]] = 1
                 matrixResult[j[2],j[1]] = -1
                
        else
                 matrixResult[j[1],j[2]] = -1
                 matrixResult[j[2],j[1]] = 1
            end
            
        end
    comparaisons[i] = matrixResult
    end
        
   return(comparaisons) 
end

    
    


computeOrderedPairs (generic function with 1 method)

In [69]:
#w1 = [0.1,0.2,0.3,0.8,1]
#Q1 = createAnswerProbabilitiesBLT(w1)
#questionsTaken1 = createAnsweredQuestions(5,5,3)
answers1 =  sampleStudentQuestionMatrix(5,3,questionsTaken1,w1)


5×5 Matrix{Float64}:
  0.0   0.0  -1.0  -1.0  -1.0
  0.0   0.0   1.0   1.0  -1.0
 -1.0  -1.0   0.0  -1.0   0.0
  1.0   0.0   1.0   0.0   1.0
  1.0   0.0   1.0   1.0   0.0

In [65]:
questionsTaken1

Dict{Any, Any} with 5 entries:
  5 => [3, 4, 1]
  4 => [1, 5, 3]
  2 => [3, 4, 5]
  3 => [1, 4, 2]
  1 => [4, 3, 5]

In [58]:
size(questionsTaken1)

(5, 5)

In [63]:
orderedComparaisons =  computeOrderedPairs(questionsTaken1,3,5,Q1)

Dict{Any, Any} with 5 entries:
  5 => [0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 0.0; … ; -1.0 0.0 … 0.0 0.0; 0.0 0.0 ……
  4 => [0.0 0.0 … 0.0 1.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; -1.0 0.0 ……
  2 => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 -1.0; 0.0 0.0 ……
  3 => [0.0 -1.0 … -1.0 0.0; 1.0 0.0 … -1.0 0.0; … ; 1.0 1.0 … 0.0 0.0; 0.0 0.0…
  1 => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 1.0; 0.0 0.0 … …

In [64]:
orderedComparaisons[1]

5×5 Matrix{Float64}:
 0.0  0.0  0.0   0.0   0.0
 0.0  0.0  0.0   0.0   0.0
 0.0  0.0  0.0  -1.0  -1.0
 0.0  0.0  1.0   0.0   1.0
 0.0  0.0  1.0  -1.0   0.0

In [ ]:
computeOrderedPairs(questions_taken,number_questions_taken,m,BLTMatrix)

In [35]:
answered[2]

3-element Vector{Int64}:
 5
 4
 3

In [34]:
 subs =  collect(subsets(answered[2],2)) 

3-element Vector{Vector{Int64}}:
 [5, 4]
 [5, 3]
 [4, 3]

In [36]:
pepe = zeros(5,5)

5×5 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

In [37]:
current = subs[1]

2-element Vector{Int64}:
 5
 4

In [39]:
pepe[current[1],current[2]]

0.0

In [32]:
answered[2]

2-element Vector{Int64}:
 4
 5

In [ ]:
 subs =  collect(answered[2],2)) 

In [66]:
dict = Dict(i => "asdf"*"$i" for i = 0:5:20)

Dict{Int64, String} with 5 entries:
  0  => "asdf0"
  5  => "asdf5"
  15 => "asdf15"
  20 => "asdf20"
  10 => "asdf10"

In [72]:
dict[5]

"asdf5"